- target-folder
    - English_EN
        - MaxiCharger AC
            - Wallbox
                - Installation and Operation Manual
                    - arch
                - Quick Guide
                    - arch
                - Pedestal EP
                    - Quick Guide
                        - arch
            - Compact
                - ...
            - Ultra
                - ...
            - Maintenance Manual
                - arch
            - Pedestal
                - Quick Guide
                    - arch
        - MaxiCharger DC
            - Fast
                - Installation and Operation Manual
                    - arch
                - Service Manual
                    - arch
                - CMS
                    - Quick Gduie
                        - arch
            - Compact
                - ...
        - Energy Management System
                - Operation and Installation Manul
                    - arch
    - German_DE
        - ...


### Test Folder Description
Under folder *"target-folder\English_EN\MaxiCharger AC\Ultra\User Manual"* there are 3 pdf-files and 1 txt-files,<br>
one of the pdf-files has unregular name and should not be identified,<br>
only the pdf-file that edited on 2023-07-06 should be verified.<br>
<br>
The document under folder *"target-folder\English_EN\MaxiCharger DC\Fast\CMS\Quick Guide"* does not exist,<br>
so that this document should be recognized as *missing*<br>
<br>
Under folder *"target-folder\French_FR\MaxiCharger AC\Wallbox\Installation and Operation Manual"* there is one pdf-files named as *"User Mnual"*,<br>
which does not comply with the naming regulation and should be recognized as *abnormal*<br>
<br>
In summary, 15 documents shall be available, but only 14 documents will be found, 1 of those is abnormal.

In [81]:
import os
import re

verif = False
doc_num = 0
fund_num = 0

for root, dirs, files in os.walk(os.path.join('target-folder', 'English_EN')):
    if "arch" in dirs:
        doc_num += 1
        print(root, "...", dirs, "...", files)
        pattern_1 = r".*\\([A-Z][a-zA-Z]+)_(\w{2})\\(MaxiCharger (AC|DC))(.*\\)(.+ (Guide|Manual))$"
        pattern_2 = r".*\\([A-Z][a-zA-Z]+)_(\w{2})\\(Energy Management System)(.*\\)(.+ (Guide|Manual))$"
        match_1 = re.match(pattern_1, root)
        match_2 = re.match(pattern_2, root)
        if match_1:
            category = match_1.group(3)
            object = match_1.group(5).replace("\\", "")
            document = match_1.group(6)    
        elif match_2:
            category = match_2.group(3)
            object = match_2.group(4).replace("\\", "")
            document = match_2.group(5)
        else:
            print("Document path '%s' not correct." % (root))
        if len(files)!=0:
            fund_num += 1
            print("Document {0} {1} {2} exists.".format(category, object, document))
        else:
            print("Document {0} {1} {2} is missing.".format(category, object, document))
    else:
        pass

print("%d documents shall be available." % (doc_num))
print("%d documents has been found." % (fund_num))

target-folder\English_EN\MaxiCharger AC\Compact\User Manual ... ['arch'] ... ['MaxiCharger AC Compact - User Manual_20230302_EN.pdf']
Document MaxiCharger AC Compact User Manual exists.
target-folder\English_EN\MaxiCharger AC\Compact\Quick Guide ... ['arch'] ... ['MaxiCharger AC Compact - Quick Guide_20230215_EN.pdf']
Document MaxiCharger AC Compact Quick Guide exists.
target-folder\English_EN\MaxiCharger AC\Ultra\User Manual ... ['arch'] ... ['MaxiCharger AC Ultra - User Manual_20230706_EN.pdf', 'MaxiCharger AC Ultra - User Manual EU_20230712_EN.txt', 'MiniCharger AC Ultra - User Manual EU_20230505_EN.pdf', 'Some - Random File_20220303_EN.pdf', 'MaxiCharger AC Ultra - User Manual EU_EN.pdf']
Document MaxiCharger AC Ultra User Manual exists.
target-folder\English_EN\MaxiCharger AC\Ultra\Quick Guide ... ['arch'] ... ['MaxiCharger AC Ultra - Quick Guide_20230330_EN.pdf']
Document MaxiCharger AC Ultra Quick Guide exists.
target-folder\English_EN\MaxiCharger AC\Wallbox\Quick Guide ... ['ar

#### Function 1: path_matcher()
The function *path_matcher()* analizes the path as argument, extract the information that the name of document under this path **MUST** contain.

In [45]:
import re

#root = 'target-folder\\English_EN\\MaxiCharger AC\\Wallbox\\Pedestal EP\\Quick Guide'
#root = 'target-folder\\English_EN\\MaxiCharger AC\\Wallbox\\Installation and Operation Manual'
#root = 'target-folder\\English_EN\\MaxiCharger AC\\Wallbox\\Pedestal EP\\'
#root = 'target-folder\\English_EN\\Energy Management System\\Operation and Installation Manual'
root = 'target-folder\\English_EN\\MaxiCharger AC\\Maintenance Manual'

def path_matcher(root:str) -> tuple[str, str, str]:
    pattern_1 = r".*\\([A-Z][a-zA-Z]+)_(\w{2})\\(MaxiCharger (AC|DC))(.*)(\\.+ (Guide|Manual))$"
    pattern_2 = r".*\\([A-Z][a-zA-Z]+)_(\w{2})\\(Energy Management System)(.*)(\\.+ (Guide|Manual))$"
    match_1 = re.match(pattern_1, root)
    match_2 = re.match(pattern_2, root)
    if match_1:
        category = match_1.group(3)
        object = match_1.group(5).replace("\\", " ").strip()
        document = match_1.group(6).replace("\\", "")     
    elif match_2:
        category = match_2.group(3)
        object = ''
        document = match_2.group(5).replace("\\", "")  
    else:
        print("Path '%s' is not correct." % (root))
        category = 'null'
        object = 'null'
        document = 'null'
    return category, object, document

t = path_matcher(root)
print(t)

('MaxiCharger AC', '', 'Maintenance Manual')


#### Function 2: name_matcher()
The function ***name matcher*** analizes the *list of files* as argument under given path. Collect name and release date of pdf-file in a *dictionary*, who's name complies the naming regulation. The pdf-file that edited recently  

In [58]:
import os
import re

from datetime import datetime

'''
# case 0
files = []
path_info = ('MaxiCharger AC', 'Ultra', 'User Manual')
'''
'''
# case 1
files = ['MaxiCharger AC Ultra - User Manual EU_EN.pdf', 
         'MaxiCharger AC Ultra - User Manual EU_20230712_EN.txt',
         'MaxiCharger AC Ultra - User Manual_20230706_EN.pdf', 
         'MiniCharger AC Ultra - User Manual EU_20230505_EN.pdf',
         'Some - Random File_20220303_EN.pdf']
root = 'target-folder\\English_EN\\MaxiCharger AC\\Ultra\\User Manual'
path_info = ('MaxiCharger AC', 'Ultra', 'User Manual')
'''
'''
# case 2
files = ['Energy Management System - Operation and Installation Manual EU_20230525_EN.pdf']
path_info = ('Energy Management System', '', 'Operation and Installation Manual')
'''
# case 3
files = ['MaxiCharger AC - Maintenance Manual EU_20220224_EN.pdf']
root = 'target-folder\\English_EN\\MaxiCharger AC\\Maintenance Manual'
path_info = ('MaxiCharger AC', '', 'Maintenance Manual')


def name_matcher (root:str, path_info:tuple[str, str, str], files:list) -> tuple[str, str]:
    # initial dict of all existing pdf files
    pdf_files = {}
    result = False
    # get formal file name
    p1, p2, p3 = path_info
    file_name = (p1 + ' ' + p2 if len(p2)>0 else p1) + ' - ' + p3
    '''
    pattern_part = r"%s %s - %s(?:.*)?.*.pdf$" % path_info
    pattern_full = r"%s %s - %s(?:.*)?_(\d{4})(\d{2})(\d{2})_(\w{2}).pdf$" % path_info
    '''
    pattern_part = file_name + '(?:.*)?.*.pdf$'
    pattern_full = file_name + '(?:.*)?_(\d{4})(\d{2})(\d{2})_(\w{2}).pdf$'
    pattern_pdf = r".*.pdf$"
    # if appropriate pdf file exists under the 
    for file in files:
        # fuzzy match
        match_part = re.match(pattern_part, file, re.IGNORECASE)
        if match_part:
            result = True
            match_full = re.match(pattern_full, file, re.IGNORECASE)
            if match_full:
                release = f"{match_full.group(1)}-{match_full.group(2)}-{match_full.group(3)}"
            else:
                release = datetime.fromtimestamp(os.path.getmtime(os.path.join(root, file))).strftime(r"%Y-%m-%d")
            pdf_files.setdefault(file, release)
        else:
            match_pdf = re.match(pattern_pdf, file)
            if match_pdf:
                print("Unregular named file: %s detected." % (file))
            else:
                print("Non-pdf file: %s detected." % (file))
            pass
    try:
        # returns full file name and its release date
        latest_file, latest_release = max(pdf_files.items(), key=lambda item: item[1])
        print(f"Latest file {latest_file} edited on {latest_release} verified.")
    except ValueError:
        latest_file = 'null'
        latest_release = 'null'
        print(f"Document {file_name} is missing.")
    return latest_file, latest_release, result

# 如果files为空，则返回null
lf, lr, r = name_matcher (root, path_info, files)
print(lf, lr, r)

Latest file MaxiCharger AC - Maintenance Manual EU_20220224_EN.pdf edited on 2022-02-24 verified.
MaxiCharger AC - Maintenance Manual EU_20220224_EN.pdf 2022-02-24 True


In [57]:
# alternative
import re

files = ['MaxiCharger AC - Maintenance Manual EU_20220224_EN.pdf',
         'Maxicharger AC - mainTenAnce ManUal Eu_20220224_eN.pdf', 
         'MaxiCharger AC - Maintenance Manual EU_20220224_EN.txt', 
         'maxicharger ac - maintenance manual EU_20220224_EN.pdf',
         'minicharger ac - user manual.pdf',
         'Some - Random File_20220303_EN.pdf']

path_info = ('MaxiCharger AC', '', 'Maintenance Manual')
p1, p2, p3 = path_info
file_name = (p1 + ' ' + p2 if len(p2)>0 else p1) + ' - ' + p3
pattern_part = file_name + '(?:.*)?.*.pdf$'
print(pattern_part)

for f in files:
    if re.match(pattern_part, f, re.IGNORECASE):
        print('yes')
    else:
        print('no')


MaxiCharger AC - Maintenance Manual(?:.*)?.*.pdf$
yes
yes
no
yes
no
no


In [22]:
d = {'MaxiCharger AC Ultra - User Manual EU_EN.pdf': '2023-06-23', 'MaxiCharger AC Ultra - User Manual_20230706_EN.pdf': '2023-07-06'}
max_key, max_value = max(d.items(), key=lambda item: item[1])
print(max_key, max_value)

MaxiCharger AC Ultra - User Manual_20230706_EN.pdf 2023-07-06


In [46]:
from datetime import datetime

current_date = datetime.now().strftime(r"%Y-%m-%d")
print(current_date)

2023-07-23


In [49]:
fs = ['f1.pdf', 'f2.exe']
print([f for f in fs if f.endswith('.pdf')])

['f1.pdf']


#### Function 3: file_extractor()
The function file extractor get the edit and release times, also the language code of the selected pdf file.

In [55]:
import os
import csv
import re

from datetime import datetime

# 目标文件夹根目录
target_folder = "target-folder"  

# 获取所有语言分类子文件夹的路径
#lang_folders = [folder for folder in os.listdir(target_folder) if os.path.isdir(os.path.join(target_folder, folder))]
#lang_folders.remove('English_EN')

lang_folder = 'English_EN'
lang_code = re.match(r"([A-Z][a-zA-Z]+)_(\w{2})", lang_folder).group(2)  # EN

csv_file = f"{lang_code}_log.csv"  # EN_log.csv
csv_path = os.path.join(target_folder, lang_folder, csv_file)

"""
"""
doc_num = 0
fund_num = 0
# 打开CSV文件并写入表头
with open(csv_path, 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(["language", "category", "object", "document", "release"])
    # 遍历目标文件夹下的所有内容
    for root, dirs, files in os.walk(os.path.join(target_folder, lang_folder)):
        # 只处理包含arch子项的文件夹
        if "arch" in dirs:
            doc_num += 1
            path_info = path_matcher(root)
            category, object, document = path_info
            file, release, result = name_matcher (root, path_info, files)
            fund_num += int(result)
            pass
        writer.writerow([lang_code, category, object, document, release])

print("%d documents shall be available." % (doc_num))
print("%d documents has been found." % (fund_num))
        


Latest file MaxiCharger AC Compact - User Manual_20230302_EN.pdf edited on 2023-03-02 verified.
Latest file MaxiCharger AC Compact - Quick Guide_20230215_EN.pdf edited on 2023-02-15 verified.
Non-pdf file: MaxiCharger AC Ultra - User Manual EU_20230712_EN.txt detected.
Unregular named file: MiniCharger AC Ultra - User Manual EU_20230505_EN.pdf detected.
Unregular named file: Some - Random File_20220303_EN.pdf detected.
Latest file MaxiCharger AC Ultra - User Manual_20230706_EN.pdf edited on 2023-07-06 verified.
Latest file MaxiCharger AC Ultra - Quick Guide_20230330_EN.pdf edited on 2023-03-30 verified.
Latest file MaxiCharger AC Wallbox - Quick Guide_20230523_EN.pdf edited on 2023-05-23 verified.
Latest file MaxiCharger AC Wallbox Pedestal EP - Quick Guide_20230512_EN.pdf edited on 2023-05-12 verified.
Latest file MaxiCharger AC Wallbox - Installation and Operation Manual v3_20230717_EN.pdf edited on 2023-07-17 verified.
Latest file MaxiCharger AC Pedestal - Quick Guide_20230223_EN.pd

#### Function 4: md_writer()
The function file extractor get the edit and release times, also the language code of the selected pdf file.

In [132]:
import os
import csv

from datetime import datetime

target_folder = "target-folder" 
lang_folders = ['English_EN']

# 生成 README.md 文件
readme_path = os.path.join(target_folder, "README.md")

with open(readme_path, "w") as readme_file:
    readme_file.write("# verification result\n\n")
    readme_file.write("# verification result\n\n")
    for lang_folder in lang_folders:
        lang_code = lang_folder.split("_")[-1]
        csv_file = f"{lang_code}_verification.csv"
        csv_path = os.path.join(lang_folder, csv_file)

        readme_file.write(f"## {lang_code}\n\n")
        readme_file.write(f"| language | category | object | document | update | status |\n")
        readme_file.write(f"| --- | --- | --- | --- | --- | --- |\n")
        readme_file.write(f"| [CSV file](./{csv_path}) |\n\n")

['f1.pdf']


In [13]:
import os

test_folder = "test-folder" 
rm_path = os.path.join(test_folder, "README.md")

values = ("v1", "v2", "v3", "v4", "v5", "v6")

with open(rm_path, "w") as rm_file:
    rm_file.write("# Titel level 1\n")
    rm_file.write("## Titel level 2\n")
    rm_file.write("### Titel level 3\n")
    rm_file.write("#### Titel level 4\n")
    rm_file.write("first line<br>\n")
    rm_file.write("second line<br>\n\n")
    rm_file.write(f"| language | category | object | document | update | status |\n")
    rm_file.write(f"| --- | --- | --- | --- | --- | --- |\n")
    for i in range(10):
        rm_file.write(f"| {values[0]} | {values[1]} | {values[2]} | {values[3]} | {values[4]} | {values[5]} |\n")
    

In [ ]:
import os

# 原始目录和目标目录
source_dir = "./test-folder/English_EN\MaxiCharger AC/Wallbox/User Manual"
target_dir = "./test-folder/English_EN\MaxiCharger AC/Wallbox/User Manual"

# 遍历源目录下的所有文件
for filename in os.listdir(source_dir):
    if filename.endswith(".pdf"):
        # 构建源文件路径和目标文件路径
        source_file = os.path.join(source_dir, filename)
        target_file = os.path.join(target_dir, filename)
        
        # 将目标文件路径中的文件名替换为指定的 filename
        target_file = os.path.join(target_dir, filename.replace(".pdf", "_" + filename))
        
        # 重命名文件
        os.rename(source_file, target_file)